In [ ]:
from sequence.kernel.timeline import Timeline
from sequence.components.memory import MemoryArray
from sequence.topology.node import Node
# cria uma instância de Timeline
runtime = 1e12  # tempo máximo de simulação em picosegundos
tl = Timeline(runtime)

# cria 
class Hub(Node):
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        # Memória de qubits, são esses argumentos
        memory_array_name = f"{self.name}.memory_array"
        # outros detalhes envolvem parametros de configuração da memória, mas por enquanto só isso
        self.memory_array = MemoryArray(memory_array_name, timeline, num_memories=10)
        # adiciona as memórias ao hub
        self.add_component(self.memory_array)
        # indica que a memória é desse hub
        self.memory_array.add_receiver(self)
    
    
hub = Hub("hub", tl)
# configuração opcional: desativar barra de progresso
tl.show_progress = False